# Convertitore Dicom To Mp4

Librerie necessarie al corretto funzionamento degli scripts:
1. [Numpy](https://pypi.org/project/numpy/)
1. [Pydicom](https://pypi.org/project/pydicom/)
1. [Os](https://pypi.org/project/OS-Platform/)
1. [Cv2](https://pypi.org/project/cv_utils/)

Prima di procedere all'importazione delle stesse è necessario procedere con l'installazione!

In [7]:
# Librerie necessarie
!pip install numpy
!pip install pydicom
!pip install opencv-python

  Using cached opencv_python-4.8.0.74-cp37-abi3-win_amd64.whl (38.1 MB)


In [8]:
# Import necessari
from glob import glob
import pydicom
import os
import cv2
from PIL import Image
import re
import numpy as np

In [9]:
# Carico il file DICOM
path=!pwd
filename = os.path.join(path[0],"sample.DCM")
ds = pydicom.dcmread(filename)
num_frames = ds.NumberOfFrames
pixel_data = ds.pixel_array
pixel_data = cv2.normalize(pixel_data, None, 0, 255, cv2.NORM_MINMAX,cv2.CV_8U)
print(pixel_data.shape)
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

(11, 430, 600)


In [10]:
# Estraggo i singoli frame immagini dal file DCM
for i in range(pixel_data.shape[0]):
    os.makedirs(path[0]+'/Output/dicom_3d_images', exist_ok=True)
    slice = pixel_data[i, :, :]
    if ds.PhotometricInterpretation == 'YBR_FULL_422':
        slice = cv2.cvtColor(slice, cv2.COLOR_YUV2RGB)
    img = Image.fromarray(slice)
    img.save(f'{path[0]}/Output/dicom_3d_images/image{i}.jpg')

In [11]:
# Includo tutti i frame in un unico video
file_path = glob(path[0]+'/Output/dicom_3d_images/*.jpg')
height, width, _ = cv2.imread(file_path[0]).shape
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 60
out = cv2.VideoWriter(path[0]+'/Output/image1.mp4', fourcc, fps, size)

for filename in sorted(file_path, key=numericalSort):
    img = cv2.imread(filename)
    out.write(img)
out.release()